In [1]:
from utils import load_data, check_gpu, check_dir
from clean_session import preprocess_sessions
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
from functools import partial
import matplotlib.pyplot as plt
from clean_session import preprocess_sessions
from create_model_inputs import prepare_data, compute_session_fts, create_model_inputs
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2
def fprint(df, name):
    print(f'{name} shape: ({df.shape[0]:,}, {df.shape[1]})')
import pprint
pp = pprint.PrettyPrinter(indent=1)

In [3]:
nrows = 1000000
train = prepare_data('train', convert_action_type=False, nrows=nrows, recompute=True)
df = compute_session_fts(train, 'train')

[05-14 19:49:27 - create_model_inputs-179 - compute_session_fts - INFO] Session features generated: ['timestamp_session_duration', 'timestamp_dwell_time_prior_clickout', 'current_filters_last_filters', 'session_id_size', 'reference_last_reference_id']


In [6]:
df = df.groupby('session_id').last().reset_index()
# df['session_id_size'] = np.log(df['session_id_size'])
# df['cfs'] = df['current_filters_last_filters'].str.lower().str.split('|')

# df['prices'] = df['prices'].str.split('|')
# df['prices'] = df['prices'].apply(lambda x: [int(p) for p in x])

# df['time_steps'] = df['prices'].str.len()
# padding_mask = df['time_steps'] < 25
# df.loc[padding_mask, 'prices'] = df.loc[padding_mask, 'prices'].apply(lambda x: np.pad(x, (0, 25-len(x)),
#                                                                                        mode='constant'))
df['imps'] = df['impressions'].str.split('|')
df['imps'] = df['imps'].apply(lambda x: [int(i) for i in x])
# df.loc[padding_mask, 'impressions'] = (df.loc[padding_mask, 'impressions']
#                                          .apply(lambda x: np.pad(x, (0, 25-len(x)),mode='constant')))
df['reference'] = df['reference'].astype(int)
# filter out nan rows with reference_id not in impressions list, since if the true target in test
# is not in the impression list then it would not get evaluated
def assign_target(row):
    ref = row['reference']
    imp = list(row['imps'])
    if ref in imp:
        return imp.index(ref)
    else:
        return np.nan

df['target'] = df.apply(assign_target, axis=1)
# drop the ones whose reference is not in the impression list
df = df[df['target'].notna()].reset_index(drop=True)
df['target'] = df['target'].astype(int)
def assign_last_ref_id(row, divide=True):
    ref = row['reference_last_reference_id']
    # although reference_id got converted to int, but the reference_last_reference_id was calculated
    # when it was still str value, so here we look up the index in str of impressions
    imp = [str(i) for i in row['imps']]

    if pd.isna(ref):
        return np.nan
    else:
        if ref in imp:
            if divide:
                return (imp.index(ref) + 1) / len(imp)
            else:
                return imp.index(ref) + 1
        else:
            return np.nan
assign_last_ref_id_func = partial(assign_last_ref_id, divide=False)
df['last_ref_ind'] = df.apply(assign_last_ref_id_func, axis=1)

### comapre session with size 1 and the rest

In [8]:
df.head(2)

,session_id,user_id,timestamp,step,action_type,current_filters,reference,impressions,prices,timestamp_session_duration,timestamp_dwell_time_prior_clickout,current_filters_last_filters,session_id_size,reference_last_reference_id,imps,target,last_ref_ind
0,000263df674fa,C2D58CUH9AOT,2018-11-05 20:36:38,5,clickout item,NaN,8249,8249|8250|14378|67361|8284|8257|1955219|8254|6...,74|88|87|72|56|87|244|168|123|50|129|72|151|83...,293.0,236.0,NaN,5,8249,"[8249, 8250, 14378, 67361, 8284, 8257, 1955219...",0,0.04
1,0002ccee5a980,M2E4246PL9XQ,2018-11-06 10:30:13,3,clickout item,NaN,109489,118737|1305086|4621782|1397441|53118|766713|92...,79|241|96|51|68|99|86|69|71|79|91|67|70|56|82|...,1.0,1.0,NaN,2,109489,"[118737, 1305086, 4621782, 1397441, 53118, 766...",19,0.80


In [9]:
ones = df[df['session_id_size']==1].reset_index(drop=True)
multiples = df[df['session_id_size']>1].reset_index(drop=True)

In [10]:
oids = ones['session_id'].unique()
mids = multiples['session_id'].unique()

---
### ones

In [170]:
display_cols = ['session_id', 'step', 'action_type', 'current_filters', 'reference', 'timestamp',
                'impressions', 'prices', 'target', 'session_id_size']
raw_display_cols = [c for c in train.columns if c in display_cols]

In [105]:
# pd.options.display.max_colwidth = 100

In [115]:
one_rid = np.random.choice(oids, 1)[0]
print(one_rid)
df[df.session_id==one_rid][display_cols].reset_index(drop=True)

ed424f52a9bd1


,session_id,step,action_type,current_filters,reference,impressions,prices,target,session_id_size
0,ed424f52a9bd1,1,clickout item,NaN,4074298,1107100|1088658|2686072|4074298|2340840|1027662|6958594|1108477|100869|350756|7200582|350946|210...,93|27|37|17|54|48|60|22|25|23|51|41|99|38|60|28|25|17|39|46|69|61|66|23|99,3,1


In [116]:
pp.pprint(df[df.session_id==one_rid][display_cols].reset_index(drop=True).to_dict(orient='list'))

{'action_type': ['clickout item'],
 'current_filters': [nan],
 'impressions': ['1107100|1088658|2686072|4074298|2340840|1027662|6958594|1108477|100869|350756|7200582|350946|2100134|100882|1704651|1553063|2430532|350751|1557355|7200206|5962152|2096974|350901|100851|392031'],
 'prices': ['93|27|37|17|54|48|60|22|25|23|51|41|99|38|60|28|25|17|39|46|69|61|66|23|99'],
 'reference': [4074298],
 'session_id': ['ed424f52a9bd1'],
 'session_id_size': [1],
 'step': [1],
 'target': [3]}


In [117]:
# and the raw
train[train.session_id==one_rid][raw_display_cols].reset_index(drop=True)

,session_id,step,action_type,current_filters,reference,impressions,prices
0,ed424f52a9bd1,1,clickout item,NaN,4074298,1107100|1088658|2686072|4074298|2340840|1027662|6958594|1108477|100869|350756|7200582|350946|210...,93|27|37|17|54|48|60|22|25|23|51|41|99|38|60|28|25|17|39|46|69|61|66|23|99


In [118]:
pp.pprint(train[train.session_id==one_rid][raw_display_cols].reset_index(drop=True).to_dict(orient='list'))

{'action_type': ['clickout item'],
 'current_filters': [nan],
 'impressions': ['1107100|1088658|2686072|4074298|2340840|1027662|6958594|1108477|100869|350756|7200582|350946|2100134|100882|1704651|1553063|2430532|350751|1557355|7200206|5962152|2096974|350901|100851|392031'],
 'prices': ['93|27|37|17|54|48|60|22|25|23|51|41|99|38|60|28|25|17|39|46|69|61|66|23|99'],
 'reference': ['4074298'],
 'session_id': ['ed424f52a9bd1'],
 'step': [1]}


---
### Longer

In [205]:
mid_rid = np.random.choice(mids, 1)[0]
# df[df.session_id==mid_rid][display_cols].reset_index(drop=True)

In [206]:
np.sort([int(i) for i in df[df.session_id==mid_rid]['prices'].str.split('|').values[0]])

array([18, 18, 22, 23, 25, 26, 26, 27, 28, 31, 32, 32, 33, 34, 36, 36, 36,
       39, 40, 40, 47, 47, 54, 59, 64])

In [207]:
# pp.pprint(df[df.session_id==mid_rid][display_cols].reset_index(drop=True).to_dict(orient='list'))

In [208]:
# the raw
train[train.session_id==mid_rid][raw_display_cols].reset_index(drop=True)

,session_id,timestamp,step,action_type,current_filters,reference,impressions,prices
0,99fd8493425de,2018-11-05 23:36:14,1,clickout item,NaN,2616758,2024547|6005436|1288340|3456506|1998987|1996107|4050726|1962445|1887447|2304486|6264470|2616758|...,26|54|34|47|32|36|32|59|39|47|31|40|64|36|40|28|27|33|36|18|39|34|44|18|36
1,99fd8493425de,2018-11-05 23:40:08,2,clickout item,NaN,2024547,2024547|4069946|478316|8265516|7936218|8299154|6793528|7851528|8122086|6005436|1288340|3456506|1...,26|28|27|18|18|26|23|25|22|54|34|47|32|36|32|59|39|47|31|40|64|36|40|33|36


In [209]:
pp.pprint(train[train.session_id==mid_rid][raw_display_cols].reset_index(drop=True).to_dict(orient='list'))

{'action_type': ['clickout item', 'clickout item'],
 'current_filters': [nan, nan],
 'impressions': ['2024547|6005436|1288340|3456506|1998987|1996107|4050726|1962445|1887447|2304486|6264470|2616758|1708747|2024615|3219674|4069946|478316|1995277|3184812|8265516|1948623|4077612|2289820|7936218|4406744',
                 '2024547|4069946|478316|8265516|7936218|8299154|6793528|7851528|8122086|6005436|1288340|3456506|1998987|1996107|4050726|1962445|1887447|2304486|6264470|2616758|1708747|2024615|3219674|1995277|3184812'],
 'prices': ['26|54|34|47|32|36|32|59|39|47|31|40|64|36|40|28|27|33|36|18|39|34|44|18|36',
            '26|28|27|18|18|26|23|25|22|54|34|47|32|36|32|59|39|47|31|40|64|36|40|33|36'],
 'reference': ['2616758', '2024547'],
 'session_id': ['99fd8493425de', '99fd8493425de'],
 'step': [1, 2],
 'timestamp': [Timestamp('2018-11-05 23:36:14'),
               Timestamp('2018-11-05 23:40:08')]}


In [186]:
# dup_mask = train[[c for c in train.columns if c not in ['timestamp', 'step']]].duplicated(keep=False)
# dup_mask.sum()

In [185]:
# dup_cols = [c for c in train.columns if c not in ['timestamp', 'step']]
# # train[dup_cols].loc[train[dup_cols].shift(-1) != train[dup_cols]]
# train.groupby(dup_cols).apply(lambda x: )

In [173]:
train.shape

(326695, 9)

In [174]:
99508/train.shape[0]

0.3045899080181821

In [2]:
a = load_data('train', nrows=1000)

[05-26 15:51:47 - utils-106 - load_data - INFO] Loading train using 1,000 rows (993 trimmed) which is 0.01% out of total train data


In [3]:
a.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [11]:
a.head(1).timestamp.diff()

0   NaN
Name: timestamp, dtype: float64